<a href="https://colab.research.google.com/github/abirhazra/Reinforcement-Learning/blob/main/DDPG_agent(Actor_Critic)_with_Keras_RL_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow
!pip install keras-rl

     |████████████████████████████████| 40kB 2.6MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp37-none-any.whl size=48380 sha256=bd0db1b9397d4d3106b30528cfefe6889905230380cf33825ccd135bc89a82c4
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl


In [3]:
import numpy as np
import gym

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Concatenate
from keras.optimizers import Adam, Adadelta, Adagrad, Adamax, SGD, RMSprop

from rl.agents import  DDPGAgent
from rl.memory import SequentialMemory
from rl.random import  OrnsteinUhlenbeckProcess

In [4]:
ENV_NAME = 'Pendulum-v0'

In [5]:
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

[123]

In [8]:
nb_actions = env.action_space.shape[0]
nb_actions

1

In [11]:
actor = Sequential()
actor.add(Flatten(input_shape = (1,) + env.observation_space.shape))
actor.add(Dense(16))
actor.add(Activation('relu'))
actor.add(Dense(16))
actor.add(Activation('relu'))
actor.add(Dense(16))
actor.add(Activation('relu'))
actor.add(Dense(nb_actions))
actor.add(Activation('linear'))
print(actor.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3)                 0         
_________________________________________________________________
dense (Dense)                (None, 16)                64        
_________________________________________________________________
activation (Activation)      (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)               

In [15]:
action_input = Input(shape=(nb_actions,),name='action_input')
observation_input = Input(shape=(1,) + env.observation_space.shape, name= 'observation_input')
flattened_observation = Flatten()(observation_input)
x = Concatenate() ([action_input, flattened_observation])
x=Dense(32)(x)
x=Activation('relu')(x)
x=Dense(32)(x)
x=Activation('relu')(x)
x=Dense(32)(x)
x=Activation('relu')(x)
x=Dense(1)(x)
x=Activation('linear')(x)
critic = Model(inputs = [action_input, observation_input], outputs = x)
print(critic.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
observation_input (InputLayer)  [(None, 1, 3)]       0                                            
__________________________________________________________________________________________________
action_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 3)            0           observation_input[0][0]          
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 4)            0           action_input[0][0]               
                                                                 flatten_4[0][0]              